# Machine Learning Foundation

## Section 2, Part C: Cross Validation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

In [3]:
import requests

def download(url, filename):
    response = requests.get(url)
    
    print(f"content {response}")
    if response.status_code == 200:
        with open(filename,'wb') as f:
            f.write(response.content)

## Data loading

In [4]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/boston_housing_clean.pickle"

download(url, 'boston_housing_clean.pickle')

content <Response [200]>


In [6]:
boston = pd.read_pickle('./boston_housing_clean.pickle')
boston.keys()

dict_keys(['dataframe', 'description'])

In [7]:
boston_data = boston['dataframe']
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Discussion: 

Suppose we want to do Linear Regression on our dataset to get an estimate, based on mean squared error, of how well our model will perform on data outside our dataset. 

Suppose also that our data is split into three folds: Fold 1, Fold 2, and Fold 3.

What would the steps be, in English, to do this?


In [8]:
y_col_name = 'MEDV'
X = boston_data.drop(y_col_name, axis=1)
y = boston_data[y_col_name]

In [9]:
kf = KFold(shuffle=True, random_state=72018, n_splits=3)

In [11]:
kf

KFold(n_splits=3, random_state=72018, shuffle=True)

In [10]:
for train_index, test_index in kf.split(X):
    print(f"train_index: {train_index[:10]}, {len(train_index)}")
    print(f"test_index: {test_index[:10]}, {len(test_index)}")
    print(f"\n")

train_index: [ 1  3  4  5  7  8 10 11 12 13], 337
test_index: [ 0  2  6  9 15 17 19 23 25 26], 169


train_index: [ 0  2  6  9 10 11 12 13 15 17], 337
test_index: [ 1  3  4  5  7  8 14 16 22 27], 169


train_index: [0 1 2 3 4 5 6 7 8 9], 338
test_index: [10 11 12 13 18 20 21 24 28 31], 168




In [ ]:
scores = []
lr = LinearRegression()

for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :],
                                        X.iloc[test_index, :],
                                        y[train_index],
                                        y[test_index])
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    score = r2_score(y_test.values, y_pred)
    scores.append(score)

scores